In [ ]:
## main.py
import functions_framework
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from keys import MySQL_pass 
from keys import API_key
import sqlalchemy

@functions_framework.http
def fetch_weather_data(request): 
    schema = "gans_cloud" # The name of cloud database
    host = "34.38.100.218"
    user = "root"
    password = MySQL_pass 
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    cities_df = pd.read_sql("cities", con=connection_string)
    
    berlin_timezone = timezone('Europe/Berlin')
    weather_items = []

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in weather_data["list"]:
            weather_item = {
                "city_id": city_id,
                "forecast_time": item.get("dt_txt"),
                "temperature": item["main"].get("temp"),
                "forecast": item["weather"][0].get("main"),
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                "wind_speed": item["wind"].get("speed"),
                "data_retrieved_at": retrieval_time
            }
            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])
    # Create SQLAlchemy engine and upload DataFrame
    engine = sqlalchemy.create_engine(connection_string)
    weather_df.to_sql("weather", con=engine, if_exists="append", index=False)
    return "Data has been updated"
    